In [1]:
import pandas as pd
import numpy as np

path = r"C:/Users/cbi7088/NCAA2020"

In [2]:
#Drop data older than 2003

tourney_result = pd.read_csv(path+'/MDataFiles_Stage1/MNCAATourneyCompactResults.csv')
tourney_seed = pd.read_csv(path+'/MDataFiles_Stage1/MNCAATourneySeeds.csv')
season_result = pd.read_csv(path+'/MDataFiles_Stage1/MRegularSeasonCompactResults.csv')

seasoncutoff = tourney_result[ tourney_result['Season'] < 2003 ].index
tourney_result.drop(seasoncutoff , inplace=True)

seasoncutoff = tourney_seed[ tourney_seed['Season'] < 2003 ].index
tourney_seed.drop(seasoncutoff , inplace=True)

seasoncutoff = season_result[ season_result['Season'] < 2003 ].index
season_result.drop(seasoncutoff , inplace=True)


In [3]:
#### Preparing Training data for tourney result ####

tourney_result = tourney_result.drop(['DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT'], axis=1)
tourney_result = pd.merge(tourney_result, tourney_seed, left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'], how='left')
tourney_result.rename(columns={'Seed':'WSeed'}, inplace=True)
tourney_result = tourney_result.drop('TeamID', axis=1)
tourney_result = pd.merge(tourney_result, tourney_seed, left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'], how='left')
tourney_result.rename(columns={'Seed':'LSeed'}, inplace=True)
tourney_result = tourney_result.drop('TeamID', axis=1)

tourney_result

,Season,WTeamID,LTeamID,WSeed,LSeed
0,2003,1421,1411,X16b,X16a
1,2003,1112,1436,Z01,Z16
2,2003,1113,1272,Z10,Z07
3,2003,1141,1166,Z11,Z06
4,2003,1143,1301,W08,W09
...,...,...,...,...,...
1110,2019,1120,1246,Y05,Y02
1111,2019,1277,1181,W02,W01
1112,2019,1403,1277,X03,W02
1113,2019,1438,1120,Z01,Y05


In [4]:


wdf = season_result.groupby(['Season','WTeamID'],as_index=False).count()
wdf = wdf.drop(['WScore','LTeamID','LScore','WLoc','NumOT'],axis=1)
wdf.rename(columns={'WTeamID':'TeamID', 'DayNum':'Wins'}, inplace=True)

ldf = season_result.groupby(['Season','LTeamID'],as_index=False).count()
ldf = ldf.drop(['LScore','WTeamID','WScore','WLoc','NumOT'],axis=1)
ldf.rename(columns={'LTeamID':'TeamID', 'DayNum':'Losses'}, inplace=True)

wl_df = pd.merge(wdf, ldf, left_on=['Season', 'TeamID'], right_on=['Season', 'TeamID'], how='left')
wl_df['WinPct'] = wl_df['Wins']/(wl_df['Wins']+wl_df['Losses'])
wl_df

,Season,TeamID,Wins,Losses,WinPct
0,2003,1102,12,16.0,0.428571
1,2003,1103,13,14.0,0.481481
2,2003,1104,17,11.0,0.607143
3,2003,1105,7,19.0,0.269231
4,2003,1106,13,15.0,0.464286
...,...,...,...,...,...
5824,2019,1462,18,15.0,0.545455
5825,2019,1463,21,7.0,0.750000
5826,2019,1464,10,20.0,0.333333
5827,2019,1465,12,14.0,0.461538


In [5]:

# Add WinPct to the tourney result and create WinPctDiff

tourney_result = pd.merge(tourney_result, wl_df, left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'], how='left')
tourney_result.rename(columns={'Wins':'WTWins', 'Losses':'WTLosses', 'WinPct':'WTWinPct'}, inplace=True)
tourney_result = pd.merge(tourney_result, wl_df, left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'], how='left')
tourney_result.rename(columns={'Wins':'LTWins', 'Losses':'LTLosses', 'WinPct':'LTWinPct'}, inplace=True)
tourney_result = tourney_result.drop(['TeamID_x','TeamID_y'],axis=1)
tourney_result['WinPctDiff'] = tourney_result['WTWinPct']-tourney_result['LTWinPct']
tourney_result

,Season,WTeamID,LTeamID,WSeed,LSeed,WTWins,WTLosses,WTWinPct,LTWins,LTLosses,LTWinPct,WinPctDiff
0,2003,1421,1411,X16b,X16a,13,16.0,0.448276,18,12.0,0.600000,-0.151724
1,2003,1112,1436,Z01,Z16,25,3.0,0.892857,19,10.0,0.655172,0.237685
2,2003,1113,1272,Z10,Z07,18,11.0,0.620690,23,6.0,0.793103,-0.172414
3,2003,1141,1166,Z11,Z06,23,6.0,0.793103,29,4.0,0.878788,-0.085684
4,2003,1143,1301,W08,W09,21,8.0,0.724138,18,12.0,0.600000,0.124138
...,...,...,...,...,...,...,...,...,...,...,...,...
1110,2019,1120,1246,Y05,Y02,25,9.0,0.735294,27,6.0,0.818182,-0.082888
1111,2019,1277,1181,W02,W01,28,6.0,0.823529,29,5.0,0.852941,-0.029412
1112,2019,1403,1277,X03,W02,26,6.0,0.812500,28,6.0,0.823529,-0.011029
1113,2019,1438,1120,Z01,Y05,29,3.0,0.906250,25,9.0,0.735294,0.170956


In [6]:
#### Preparing training data to make seeds int ####

def get_seed(x):
    return int(x[1:3])

tourney_result['WSeed'] = tourney_result['WSeed'].map(lambda x: get_seed(x))
tourney_result['LSeed'] = tourney_result['LSeed'].map(lambda x: get_seed(x))

In [7]:
#### Preparing training data ####
#Split season result into df for all game wins, df for all game losses, then concat dfs to show total points scored in season

season_win_result = season_result[['Season', 'WTeamID', 'WScore']]
season_lose_result = season_result[['Season', 'LTeamID', 'LScore']]
season_win_result.rename(columns={'WTeamID':'TeamID', 'WScore':'Score'}, inplace=True)
season_lose_result.rename(columns={'LTeamID':'TeamID', 'LScore':'Score'}, inplace=True)
season_result = pd.concat((season_win_result, season_lose_result)).reset_index(drop=True)
season_score = season_result.groupby(['Season', 'TeamID'])['Score'].sum().reset_index()

C:\Users\cbi7088\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [8]:
season_score

,Season,TeamID,Score
0,2003,1102,1603
1,2003,1103,2127
2,2003,1104,1940
3,2003,1105,1866
4,2003,1106,1781
...,...,...,...
5829,2019,1462,2372
5830,2019,1463,2265
5831,2019,1464,2205
5832,2019,1465,1962


In [9]:
season_score = pd.merge(season_score, wl_df, left_on=['Season','TeamID'], right_on=['Season','TeamID'], how='inner')
season_score

,Season,TeamID,Score,Wins,Losses,WinPct
0,2003,1102,1603,12,16.0,0.428571
1,2003,1103,2127,13,14.0,0.481481
2,2003,1104,1940,17,11.0,0.607143
3,2003,1105,1866,7,19.0,0.269231
4,2003,1106,1781,13,15.0,0.464286
...,...,...,...,...,...,...
5824,2019,1462,2372,18,15.0,0.545455
5825,2019,1463,2265,21,7.0,0.750000
5826,2019,1464,2205,10,20.0,0.333333
5827,2019,1465,1962,12,14.0,0.461538


In [10]:
#### Preparing training data ####
# Merging tourney result df with season score df

tourney_result = pd.merge(tourney_result, season_score, left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'], how='left')
tourney_result.rename(columns={'Score':'WScoreT'}, inplace=True)
tourney_result = tourney_result.drop('TeamID', axis=1)
tourney_result = pd.merge(tourney_result, season_score, left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'], how='left')
tourney_result.rename(columns={'Score':'LScoreT'}, inplace=True)
tourney_result = tourney_result.drop(['TeamID','Wins_y','Losses_y','WinPct_y'], axis=1)

where_are_NaNs = np.isnan(tourney_result)
tourney_result[where_are_NaNs] = .01

tourney_result

,Season,WTeamID,LTeamID,WSeed,LSeed,WTWins,WTLosses,WTWinPct,LTWins,LTLosses,LTWinPct,WinPctDiff,WScoreT,Wins_x,Losses_x,WinPct_x,LScoreT
0,2003,1421,1411,16,16,13,16.0,0.448276,18,12.0,0.600000,-0.151724,2065,13,16.0,0.448276,2184
1,2003,1112,1436,1,16,25,3.0,0.892857,19,10.0,0.655172,0.237685,2386,25,3.0,0.892857,1966
2,2003,1113,1272,10,7,18,11.0,0.620690,23,6.0,0.793103,-0.172414,2203,18,11.0,0.620690,2161
3,2003,1141,1166,11,6,23,6.0,0.793103,29,4.0,0.878788,-0.085684,2301,23,6.0,0.793103,2615
4,2003,1143,1301,8,9,21,8.0,0.724138,18,12.0,0.600000,0.124138,2160,21,8.0,0.724138,2172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,2019,1120,1246,5,2,25,9.0,0.735294,27,6.0,0.818182,-0.082888,2682,25,9.0,0.735294,2532
1111,2019,1277,1181,2,1,28,6.0,0.823529,29,5.0,0.852941,-0.029412,2680,28,6.0,0.823529,2839
1112,2019,1403,1277,3,2,26,6.0,0.812500,28,6.0,0.823529,-0.011029,2339,26,6.0,0.812500,2680
1113,2019,1438,1120,1,5,29,3.0,0.906250,25,9.0,0.735294,0.170956,2299,29,3.0,0.906250,2682


In [11]:
#### Preparing training data ####
# New df containing Seed Diff, Score Diff, and tourney game result
# Also concats game data into winning and losing (doubling amount of data)

tourney_win_result = tourney_result.drop(['Season', 'WTeamID', 'LTeamID','Wins_x','Losses_x','WinPct_x'], axis=1)
tourney_win_result.rename(columns={'WSeed':'Seed1', 'LSeed':'Seed2', 'WScoreT':'ScoreT1', 'LScoreT':'ScoreT2'}, inplace=True)

tourney_lose_result = tourney_win_result.copy()
tourney_lose_result['Seed1'] = tourney_win_result['Seed2']
tourney_lose_result['Seed2'] = tourney_win_result['Seed1']
tourney_lose_result['ScoreT1'] = tourney_win_result['ScoreT2']
tourney_lose_result['ScoreT2'] = tourney_win_result['ScoreT1']

tourney_win_result['Seed_diff'] = tourney_win_result['Seed1'] - tourney_win_result['Seed2']
tourney_win_result['ScoreT_diff'] = tourney_win_result['ScoreT1'] - tourney_win_result['ScoreT2']
tourney_lose_result['Seed_diff'] = tourney_lose_result['Seed1'] - tourney_lose_result['Seed2']
tourney_lose_result['ScoreT_diff'] = tourney_lose_result['ScoreT1'] - tourney_lose_result['ScoreT2']

tourney_win_result['result'] = 1
tourney_lose_result['result'] = 0
tourney_result = pd.concat((tourney_win_result, tourney_lose_result)).reset_index(drop=True)

tourney_result

,Seed1,Seed2,WTWins,WTLosses,WTWinPct,LTWins,LTLosses,LTWinPct,WinPctDiff,ScoreT1,ScoreT2,Seed_diff,ScoreT_diff,result
0,16,16,13,16.0,0.448276,18,12.0,0.600000,-0.151724,2065,2184,0,-119,1
1,1,16,25,3.0,0.892857,19,10.0,0.655172,0.237685,2386,1966,-15,420,1
2,10,7,18,11.0,0.620690,23,6.0,0.793103,-0.172414,2203,2161,3,42,1
3,11,6,23,6.0,0.793103,29,4.0,0.878788,-0.085684,2301,2615,5,-314,1
4,8,9,21,8.0,0.724138,18,12.0,0.600000,0.124138,2160,2172,-1,-12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2225,2,5,25,9.0,0.735294,27,6.0,0.818182,-0.082888,2532,2682,-3,-150,0
2226,1,2,28,6.0,0.823529,29,5.0,0.852941,-0.029412,2839,2680,-1,159,0
2227,2,3,26,6.0,0.812500,28,6.0,0.823529,-0.011029,2680,2339,-1,341,0
2228,5,1,29,3.0,0.906250,25,9.0,0.735294,0.170956,2682,2299,4,383,0


In [12]:
#### Create model ####

import matplotlib.pyplot as plt  

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline

In [13]:
#### Create model ####
# Create train and test set from training data

seed = 12345

scaler = StandardScaler()
#scaled_data = scaler.fit_transform(tourney_result)

X = tourney_result.drop('result', axis=1)
y = tourney_result.result

X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= seed)

In [14]:
tourney_result

,Seed1,Seed2,WTWins,WTLosses,WTWinPct,LTWins,LTLosses,LTWinPct,WinPctDiff,ScoreT1,ScoreT2,Seed_diff,ScoreT_diff,result
0,16,16,13,16.0,0.448276,18,12.0,0.600000,-0.151724,2065,2184,0,-119,1
1,1,16,25,3.0,0.892857,19,10.0,0.655172,0.237685,2386,1966,-15,420,1
2,10,7,18,11.0,0.620690,23,6.0,0.793103,-0.172414,2203,2161,3,42,1
3,11,6,23,6.0,0.793103,29,4.0,0.878788,-0.085684,2301,2615,5,-314,1
4,8,9,21,8.0,0.724138,18,12.0,0.600000,0.124138,2160,2172,-1,-12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2225,2,5,25,9.0,0.735294,27,6.0,0.818182,-0.082888,2532,2682,-3,-150,0
2226,1,2,28,6.0,0.823529,29,5.0,0.852941,-0.029412,2839,2680,-1,159,0
2227,2,3,26,6.0,0.812500,28,6.0,0.823529,-0.011029,2680,2339,-1,341,0
2228,5,1,29,3.0,0.906250,25,9.0,0.735294,0.170956,2682,2299,4,383,0


In [15]:
#### Create model ####
# Pick a model

model = LogisticRegression(random_state=seed)
#model = RandomForestRegressor(random_state = seed)

model.fit(X_train, y_train)
#predictions = model.predict_proba(X_test)[:, 1]
predictions = model.predict(X_test)

In [16]:
#### Create model ####

print('X_train shape: ',X_train.shape)
print('y_train shape: ',y_train.shape)
print('X_test shape: ',X_test.shape)
print('y_test shape: ',y_test.shape)
print('predictions shape: ', predictions.shape)
print('Model Score: ', model.score(X_test, y_test))
#print(model.coef_)
#print(tourney_result.columns)

X_train shape:  (1784, 13)
y_train shape:  (1784,)
X_test shape:  (446, 13)
y_test shape:  (446,)
predictions shape:  (446,)
Model Score:  0.7107623318385651


In [17]:
t_list = tourney_result.columns
t_list = t_list[:-1]

dfObj = pd.DataFrame(model.coef_, columns = t_list).T.abs().sort_values(by=0, ascending=False)
dfObj

,0
Seed_diff,0.499581
Seed1,0.419764
Seed2,0.373349
WinPctDiff,0.170429
WTWinPct,0.141680
ScoreT_diff,0.108818
LTLosses,0.099718
ScoreT1,0.079778
ScoreT2,0.064791
WTLosses,0.043266


In [18]:
from sklearn import model_selection
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

kfold = model_selection.KFold(n_splits=10, random_state=seed)
scoring = 'accuracy'
results = model_selection.cross_val_score(model, X, y, cv=kfold, scoring=scoring)
print('CV scoring: ', 'Mean: ', results.mean().round(3), 'Stdev: ', results.std().round(3))

matrix = confusion_matrix(y_test, predictions)
print(matrix)

report = classification_report(y_test, predictions,digits=3)
print(report)

CV scoring:  Mean:  0.563 Stdev:  0.07
[[146  59]
 [ 70 171]]
              precision    recall  f1-score   support

           0      0.676     0.712     0.694       205
           1      0.743     0.710     0.726       241

    accuracy                          0.711       446
   macro avg      0.710     0.711     0.710       446
weighted avg      0.712     0.711     0.711       446

